In [ ]:
# https://freeprog.tistory.com/329
#update 7 버전에서는 이미 있는 엑셀의 수정이 용이하도록 순서에 따른 로직이 아니라 리스트 검색에 따른 로직으로 작성함.
import sys
import IPython
import imutils
import cv2
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse

import xlwings as xw
import os
import subprocess

QApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True)



class Left(QWidget): 
    w= 0
    h= 0
    pixelsPerMetric=0
    fname= ''
    finame = ''
    sflist = []
    dir1= ''
    num = 0
    i= 0

    dB0=0
    exnum = 0
    exshadow=[]
    width = 10
    xl_updated_last=""

    def __init__(self):
    
        super().__init__()
           
#         astm=cv2.imread('C:/Users/Hojin/Anaconda3/envs/tensorflow/MYY/Measure/mouse_tracking/images/astm_pit.jpg')
#         cv2.imshow("ASTM",astm )
#         # cv2.waitKey(0)

        
        # 전체 폼 박스
        self.formbox = QHBoxLayout()
        self.setLayout(self.formbox)

        # 좌, 우 레이아웃박스
        self.left = QVBoxLayout()
        self.middle = QVBoxLayout()

        # 그룹박스1 생성 및 좌 레이아웃 배치
        gb = QGroupBox('그리기 종류')        
        self.left.addWidget(gb)

        # 그룹박스1 에서 사용할 레이아웃
        box = QVBoxLayout()
        gb.setLayout(box)        

        # 그룹박스 1 의 라디오 버튼 배치
        text = ['horizon']
        self.radiobtns = []

        for i in range(len(text)):
            self.radiobtns.append(QRadioButton(text[i], self))
            self.radiobtns[i].clicked.connect(self.radioClicked)
            box.addWidget(self.radiobtns[i])

        self.radiobtns[0].setChecked(True)
        self.drawType = 0
         
        # 그룹박스2
        gb = QGroupBox('펜 설정')        
        self.left.addWidget(gb)        

        grid = QGridLayout()      
        gb.setLayout(grid)        

        label = QLabel('선굵기')
        grid.addWidget(label, 0, 0)

        self.combo = QComboBox()
        grid.addWidget(self.combo, 0, 1)       

        for i in range(1, 21):
            self.combo.addItem(str(i))

        label = QLabel('선색상')

        grid.addWidget(label, 1,0)        
         
        self.pencolor = QColor(0,0,0)
        self.penbtn = QPushButton()        
        self.penbtn.setStyleSheet('background-color: rgb(0,0,0)')
        self.penbtn.clicked.connect(self.showColorDlg)
        grid.addWidget(self.penbtn,1, 1)
         

        # 그룹박스3
        gb = QGroupBox('붓 설정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)

        label = QLabel('붓색상')
        hbox.addWidget(label)                

        self.brushcolor = QColor(255,255,255)
        self.brushbtn = QPushButton()        
        self.brushbtn.setStyleSheet('background-color: rgb(255,255,255)')
        self.brushbtn.clicked.connect(self.showColorDlg)
        hbox.addWidget(self.brushbtn)
        
        
        # 그룹박스4
        gb = QGroupBox('불러오기')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)    
            
        self.openbtn = QPushButton("File Open")
        self.openbtn.clicked.connect(self.openbtnClicked)
        self.label = QLabel()

        hbox.addWidget(self.openbtn)
        hbox.addWidget(self.label)
        
         # 그룹박스5
        gb = QGroupBox('불러오기 방법')        
        self.left.addWidget(gb)

        box2 = QHBoxLayout()
        gb.setLayout(box2)        

        text = ['*.확장자(unstack)', '개별(unstack)']
        self.radiobtns2 = []

        for i in range(len(text)):
            self.radiobtns2.append(QRadioButton(text[i], self))
            self.radiobtns2[i].clicked.connect(self.radio2Clicked)
            box2.addWidget(self.radiobtns2[i])

        self.radiobtns2[0].setChecked(True)
        self.loadType = 0
         
    
        #그룹박스 6
        gb=QGroupBox('처리할 목록')
        self.left.addWidget(gb)
        
        grid2= QGridLayout()
        gb.setLayout(grid2)
        
        
        self.tb = QTextBrowser()
        self.tb.setAcceptRichText(True)
        self.tb.setOpenExternalLinks(True)

        self.previous_btn = QPushButton('previous')
        self.previous_btn.pressed.connect(self.previous_view)
        self.next_btn = QPushButton('Next')
        self.next_btn.pressed.connect(self.next_view)
        self.clear_btn = QPushButton('Clear')
        self.clear_btn.pressed.connect(self.clear_text)

        self.lbl_000 = QLabel("  ")
        
        grid2.addWidget(self.lbl_000,0,0)
        grid2.addWidget(self.tb, 1,0)
        grid2.addWidget(self.clear_btn, 2,0)
        grid2.addWidget(self.previous_btn,3,0)
        grid2.addWidget(self.next_btn, 4,0)
        
        
        # 그룹박스7
        gb = QGroupBox('스케일 바')        
        
        self.left.addWidget(gb)
        grid3=QGridLayout()
        gb.setLayout(grid3)
        # LineEdit
        self.lineEdit = QLineEdit("", self)
#         self.lineEdit.returnPressed.connect(self.lineEditChanged)
        self.lineEdit.textChanged.connect(self.lineEditChanged)
        grid3.addWidget(self.lineEdit,0,0)
        self.statusBar = QStatusBar(self)
        
#         self.lbl_bar=QLabel("  ")
        grid3.addWidget(self.statusBar,0,1)
        
        
        
        # 그룹박스7
        gb = QGroupBox('스케일')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.scalebtn = QPushButton()  
        self.scalebtn.clicked.connect(self.scalepnt)
        
        hbox.addWidget(self.scalebtn)
        self.scalebtn.setText('print')
        
        self.lbl_1 = QLabel("  ")
        hbox.addWidget(self.lbl_1)
        
        
        # 그룹박스8
        gb = QGroupBox('크기 측정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.pixelpntbtn = QPushButton()  
        self.pixelpntbtn.clicked.connect(self.pixelpnt)
        
        hbox.addWidget(self.pixelpntbtn)
        self.pixelpntbtn.setText('print')
        
        self.lbl_2 = QLabel("  ")
        hbox.addWidget(self.lbl_2)
        
        #
        gb = QGroupBox('공식 적정 판단')        
        self.left.addWidget(gb)        

        grid = QGridLayout()      
        gb.setLayout(grid)       
        
        label_d = QLabel('pit 판단')
        grid.addWidget(label_d, 0, 0)

        self.combo_d = QComboBox()
        grid.addWidget(self.combo_d, 0, 1)       
        self.decision=('','Great','normal','bad','worst','concious')
        for i in range(0,6):
            self.combo_d.addItem(str(self.decision[i]))
        
        label_d2 = QLabel('pit 형태 판단')
        grid.addWidget(label_d2, 1, 0)

        self.combo_d2 = QComboBox()
        grid.addWidget(self.combo_d2, 1, 1)       
        self.decision2=('','A','B','B,C','C','D','E','F(hori)','F(Verti)')
        for i in range(0,9):
            self.combo_d2.addItem(str(self.decision2[i]))
        
        self.combo_d.currentIndexChanged.connect(self.combo_d_c)
        self.combo_d2.currentIndexChanged.connect(self.combo_d2_c)
        
         # 그룹박스9
        gb = QGroupBox('엑셀')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.excelbtn = QPushButton()  
        self.excelbtn.clicked.connect(self.excelup)
        
        hbox.addWidget(self.excelbtn)
        self.excelbtn.setText('엑셀업댓')
        
        self.lbl_3 = QLabel("  ")
        hbox.addWidget(self.lbl_3)
        
        self.left.addStretch(1)
        
        
        
        # 전체 폼박스에 왼쪽 박스 배치
        self.formbox.addLayout(self.left)
        self.formbox.setStretchFactor(self.left, 0)

        self.openbtnClicked()#############파일 선택
        
    def combo_d_c(self):
        self.cd=self.combo_d.currentText()
           
    def combo_d2_c(self):
        self.cd2=self.combo_d2.currentText()
        
    def radioClicked(self):
        for i in range(len(self.radiobtns)):
            if self.radiobtns[i].isChecked():
                self.drawType = i                
                break

    def checkClicked(self):
        pass
             
    def showColorDlg(self):       
         
        # 색상 대화상자 생성      
        color = QColorDialog.getColor()

        sender = self.sender()

        # 색상이 유효한 값이면 참, QFrame에 색 적용
        if sender == self.penbtn and color.isValid():           
            self.pencolor = color
            self.penbtn.setStyleSheet('background-color: {}'.format( color.name()))
        else:
            self.brushcolor = color
            self.brushbtn.setStyleSheet('background-color: {}'.format( color.name()))

            

    def clear_text(self) :

        self.tb.clear()

        
    def radio2Clicked(self):
        for i in range(len(self.radiobtns2)):
            if self.radiobtns2[i].isChecked():
                self.loadType = i                
                break
        
    
    def openbtnClicked(self):
        
        
        self.tuplefname = QFileDialog.getOpenFileName(self)
        self.fname = self.tuplefname[0]
        print(self.fname)
        dir,file = os.path.split(self.fname)
        print("1 ",dir, "에 있는 ", file)
#         상대경로로 쓰려고 할때 작업공간 계속 바꿔줬는데, 절대경로로 불러오면 안씀.
#   같은 위치 디렉토리 설정 에러 피하기 위해 동일한 디렉토리로는 변경하지 않음.
        self.dir1=dir
        if self.dir1==Left.dir1:
            None
        else:
            os.chdir(dir)
        Left.dir1=self.dir1
        
        print("현재 작업공간은 ",os.getcwd())
        self.label.setText(file)
        self.finame = file
        self.foname =os.path.basename(dir)
        self.flist = os.listdir(dir)
        print("2 ", self.flist)
        print("3 ", os.getcwd())
        
        #선택: 클릭한 확장자의 파일명을 클릭한 파일이 있는 폴더 내에서 모두 찾아 리스트 저장하기.(초기화 후)
        #선택: 클릭한 파일만 리스트에 저장하기(초기화 후)
        #텍스트브라우저에 나타내기
        search = os.path.splitext(file)[1]
        print("4 확장자는 ",search)
        
        self.sflist =[]
        self.tb.clear()
        if self.loadType == 0:
            
            
            for name in self.flist:
                if search in name:
                    self.sflist.append(name)
                    print("5 처리할 데이터 목록은 ", self.sflist)
                    self.tb.append(name)
                else:
                    None
            Left.sflist= self.sflist
                    
        else:
            self.sflist.append(file)
            print("5 처리할 데이터는 ", self.sflist)
            self.tb.append(file)
            
        Left.sflist = self.sflist
        Left.fname = self.fname
        Left.finame = self.finame
        #원래의 Graview내용
        print("Graview안에 있는 w,h는 ", self.w, self.h)
        Left.i = 0
        Left.changeView(self)
        
        
        
        
    def changeView(self):
        if Left.num ==0:
            None
        else:
            Left.deleteItems(self.middle)
#             self.view.deleteLater()
#             self.view = None
        
        self.view =CView(self)
        
        (Left.w,Left.h,Left.pixelsPerMetric,Left.width,Left.dB0)=ppp.ppp(Left.sflist[Left.i])
        
        Left.h = self.h
        Left.w = self.w
        
        
        self.view.setFixedHeight(self.h)
        self.view.setFixedWidth(self.w)
        self.middle.addWidget(self.view)
         # 전체 폼박스에 오른쪽 박스 배치
        self.formbox.addLayout(self.middle)

        self.formbox.setStretchFactor(self.middle, 0)
        print("이번에 띄울 이미지는 ",Left.i,"번째 이미지")
        self.setWindowTitle(self.sflist[Left.i])
        self.setGeometry(5,30,self.w+200,self.h)

        Left.scalepnt(self)
        self.lbl_2.setText(None)
        Left.num +=1
        
        self.lbl000="처리할 개수는 : "+str(len(self.sflist))+"개, "+"현재 "+str(Left.i+1)+"번째"
        self.lbl_000.setText(self.lbl000)
        self.lbl_000.adjustSize()
        
        self.lbl_3.setText("")
        self.lbl_3.adjustSize()
        
    
    def deleteItems(layout): 
        if layout is not None: 
            while layout.count(): 
                item = layout.takeAt(0)
                print(item)
                widget = item.widget() 

                if widget is not None: 
                    widget.deleteLater() 
                else: 
                    deleteItems(item.layout()) 

                    
    def previous_view(self):
        if Left.i ==0:
            None
        else: 
            Left.i-=1
        Left.changeView(self)
                    
                    
                
    def next_view(self):
        if Left.i ==len(self.sflist)-1:
            None
        else: 
            Left.i+=1
        Left.changeView(self)
    
    
    def lineEditChanged(self):
        Left.width=int(self.lineEdit.text())
        self.statusBar.showMessage(self.lineEdit.text()+" "+chr(956)+"m") 
    
    
    
    def scalepnt(self):
        global dy
#         global pixelsPerMetric
        
        
        self.txt_1=repr("{} ".format(Left.width)+chr(956)+"m")+"길이당 {:}".format(Left.dB0)+" pixels"    
        
        
        self.lbl_1.setText(self.txt_1)
        self.lbl_1.adjustSize()   
            
    def pixelpnt(self):
        global dy
#         global pixelsPerMetric
        Left.pixelsPerMetric = Left.dB0 / Left.width
        print(Left.pixelsPerMetric)
        
        self.size= dy/Left.pixelsPerMetric
        
        self.txt_2="{}픽셀, ".format(dy)+repr("{:.5f} ".format(self.size)+chr(956)+"m")     
        
        
        self.lbl_2.setText(self.txt_2)
        self.lbl_2.adjustSize()    


    def excelup(self):
        #https://docs.xlwings.org/en/stable/quickstart.html
        folder=self.foname+"_xlsx_Pitdepth_data"
        current_directory = os.getcwd()
        
        #디렉터리의 존재 여부를 확인하고, 없을 경우 해당 디렉터리를 생성
        
        try:
            if not(os.path.isdir(current_directory + os.sep+  folder)):
                os.makedirs(os.path.join(folder))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print("Failed to create directory!!!!!")
                raise
        
        if os.path.isfile(current_directory + os.sep+  folder + os.sep + "pit_depth_result.xlsx"):
            wb = xw.Book(current_directory + os.sep+  folder + os.sep + "pit_depth_result.xlsx")
        else:
            wb = xw.Book()
        
        #이름이 있는 시트를 생성
        
        #(self.sflist[i],Left.width,self.size,self.foname)
        #Sheet1에다 입력
        sht1 = wb.sheets[0]
        sht1.range('A1').value = ['start', '','']
        sht1.range('A2').value = ['Data_name', 'scalebar ('+chr(956)+"m)", 'Pit_depth ('+chr(956)+"m)","Quality","Pit Shape"]        
        self.exshadow = []
        self.rownum = sht1.range('A1').end('down').last_cell.row
        print('rownum',self.rownum)
        for i in range(self.rownum-1):
            print(sht1.range('A'+str(i+2)).value)
            self.exshadow.append(sht1.range('A'+str(i+2)).value)
        
        print('exshadow0',self.exshadow)
        if self.sflist[Left.i] not in self.exshadow:
            self.sfn=self.sflist[Left.i]
            self.exshadow.append(self.sfn)
            print('exshadow1',self.exshadow)
            
            sht1.range('A'+str(self.rownum+1)).value = [self.sflist[Left.i], Left.width, self.size,self.cd, self.cd2 ]
        else:

            exn=self.exshadow.index(self.sflist[Left.i])+1
            print('exe=',exn)
            sht1.range('A'+str(exn+1)).value = [self.sflist[Left.i], Left.width, self.size, self.cd, self.cd2 ]
        #저장할 디렉토리를 알기위해, 현재의 디렉토리를 가져온다
        
        #현재디렉토리 아래의 xlsx_data라는 폴더에 작업한 엑셀파일을 저장
        #os.sep는 리눅스, 유니스 계열인 경우 슬래쉬, 윈도우 에서는 역슬래쉬 로 표시됨
        wb.save(current_directory + os.sep+  folder + os.sep + "pit_depth_result.xlsx")
        #time.sleep(0.5)
#         wb.close()
        self.lbl_3.setText(" 완료 ")
        self.lbl_3.adjustSize()


# QGraphicsView display QGraphicsScene
class CView(QGraphicsView):
    pics =[]
    num = 0
    def __init__(self, parent):
        
        super().__init__(parent)   
        
        self.dat= [Left.w,Left.h]
        print("CView로 받은 w,h 는 ",self.dat[:])
        
        print("CView에서의 작업공간은 ",os.getcwd())
        print("파일명은",Left.sflist[Left.i])
        
        
        self.scene = QGraphicsScene()
        

        pic = QPixmap('./'+Left.sflist[Left.i])

        self.scene.addPixmap(pic)
        self.setScene(self.scene)
        

        
        self.items = []
        
        self.start = QPointF()
        self.end = QPointF()
        self.setRenderHint(QPainter.HighQualityAntialiasing)  
        
#     @staticmethod    
#     def getwhfname(w,h,fname):
        
#         return w, h, fname
        
    def moveEvent(self, e): # 없애면 스크롤바 생김
        rect = QRectF(self.rect())
        rect.adjust(0,0,-2,-2)

        self.scene.setSceneRect(rect)
    
    def mousePressEvent(self, e):
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])

        if e.button() == Qt.LeftButton:
            # 시작점 저장
            self.start = e.pos()
            self.end = e.pos() 
            
    
    def mouseMoveEvent(self, e):  
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])
                
        # e.buttons()는 정수형 값을 리턴, e.button()은 move시 Qt.Nobutton 리턴 
        if e.buttons() & Qt.LeftButton:           

            self.end = e.pos()

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())

        
            # 수평선 그리기
            if self.parent().drawType == 0:
                 
                # 장면에 그려진 이전 선을 제거            
                if len(self.items) > 0:
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                
                hori1 = QLineF(0, self.start.y(), Left.w, self.start.y())
                
                self.items.append(self.scene.addLine(hori1, pen))
                
                hori2 = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                
                self.items.append(self.scene.addLine(hori2, pen))
                print(self.start.y(),self.end.y())
                
                
                
                """
                # 현재 선 추가
                hori = QLineF(self.start.x(), self.start.y(), self.end.x(), self.start.y())                
                self.items.append(self.scene.addLine(hori, pen))
                """

                
                
                
                
            
    
    def mouseReleaseEvent(self, e):  
        
        if e.button() == Qt.RightButton:
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                #del(self.items[-1])    

        if e.button() == Qt.LeftButton:

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())
            global dy 
            dy = abs(self.start.y()-self.end.y())
            self.dx = abs(self.start.x()-self.end.x())
            print(dy, self.dx)
            

    """
            if self.parent().drawType == 0:
                
                self.items.clear()
                hori = QLineF(0, self.start.y(), w, self.start.y())
                self.scene.addLine(hori, pen)
                hori = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                self.scene.addLine(hori, pen)
                print(self.start,self.end)
    """  
    
    
class ppp():
    
    @staticmethod
    def midpoint(ptA, ptB):
        return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)  
    @staticmethod
    def ppp(fname):
    
       
        # global fname
    #     fname = './images/example_04.bmp'
        width=Left.width





        # load the image, convert it to grayscale, and blur it slightly
        image =cv2.imread(fname)
#         cv2.imshow('image',image)
        h, w = image.shape[:2]
        cv2.waitKey(0)
        h1=round(0.938*h)
        h2=round(0.97*h)
#         w2=round(0.691*w)
        w2=round(0.65*w)
        print(h1, h2, w2)
        image = image[h1:h2,0:w2]


        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        blur = cv2.GaussianBlur(gray, (7, 7), 0)

        # perform edge detection, then perform a dilation + erosion to
        # close gaps in between object edges
        canny = cv2.Canny(blur, 50, 100)
        edged = cv2.dilate(canny, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)

        # find contours in the edge map
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        cnts = imutils.grab_contours(cnts)

        # sort the contours from left-to-right and initialize the
        # 'pixels per metric' calibration variable
        (cnts, _) = contours.sort_contours(cnts)
    #     global pixelsPerMetric
        pixelsPerMetric = None

        # loop over the contours individually
        for c in cnts:
            # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(c) < 100:
                continue

            # compute the rotated bounding box of the contour
            orig = image.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            #print(box)

            # order the points in the contour such that they appear
            # in top-left, top-right, bottom-right, and bottom-left
            # order, then draw the outline of the rotated bounding
            # box
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

            # unpack the ordered bounding box, then compute the midpoint
            # between the top-left and top-right coordinates, followed by
            # the midpoint between bottom-left and bottom-right coordinates
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = ppp.midpoint(tl, tr)
            (blbrX, blbrY) = ppp.midpoint(bl, br)

            # compute the midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-righ and bottom-right
            (tlblX, tlblY) = ppp.midpoint(tl, bl)
            (trbrX, trbrY) = ppp.midpoint(tr, br)

            # draw the midpoints on the image
            cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

            # draw lines between the midpoints
            cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                (255, 0, 255), 2)
            cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                (255, 0, 255), 2)

            # compute the Euclidean distance between the midpoints
            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            
            if pixelsPerMetric is None:
                pixelsPerMetric = dB / width
                dB0=dB
            # compute the size of the object
            dimA = dA / pixelsPerMetric
            dimB = dB / pixelsPerMetric

            # draw the object sizes on the image
            cv2.putText(orig, "{:.5f}".format(dimB),
                (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)
            cv2.putText(orig, "{:.5f}".format(dimA),(int(trbrX + 10), int(trbrY)),
                cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)   

            ##cv2.imshow("Image", orig)
            ##cv2.waitKey(0)
            ##cv2.destroyAllWindows()
        return w,h, pixelsPerMetric, width,dB0


if __name__=="__main__":
    app = QApplication(sys.argv)
    ex = Left()
    ex.show()
    sys.exit(app.exec_())
cv2.destroyAllWindows()


C:/Users/Hojin/Desktop/부식/2019 부식/19년도 부식실험 - FE-SEM 이미지/190530 - 2500H Shot-peening/190530-2/190530-2/8-13 down_15kV PC6 WD10_15.bmp
1  C:/Users/Hojin/Desktop/부식/2019 부식/19년도 부식실험 - FE-SEM 이미지/190530 - 2500H Shot-peening/190530-2/190530-2 에 있는  8-13 down_15kV PC6 WD10_15.bmp
현재 작업공간은  C:\Users\Hojin\Desktop\부식\2019 부식\19년도 부식실험 - FE-SEM 이미지\190530 - 2500H Shot-peening\190530-2\190530-2
2  ['190530-2_xlsx_Pitdepth_data', '8-13 down_15kV PC6 WD10_15.bmp', '8-13 down_15kV PC6 WD10_15.txt', '8-13 down_15kV PC6 WD10_16.bmp', '8-13 down_15kV PC6 WD10_16.txt', '8-13 down_15kV PC6 WD10_17.bmp', '8-13 down_15kV PC6 WD10_17.txt', '8-13 down_15kV PC6 WD10_18.bmp', '8-13 down_15kV PC6 WD10_18.txt', '8-13 down_15kV PC6 WD10_19.bmp', '8-13 down_15kV PC6 WD10_19.txt', '8-13 down_15kV PC6 WD10_20.bmp', '8-13 down_15kV PC6 WD10_20.txt', '8-13 down_15kV PC6 WD10_21.bmp', '8-13 down_15kV PC6 WD10_21.txt', '8-13 down_15kV PC6 WD10_22.bmp', '8-13 down_15kV PC6 WD10_22.txt', '8-13 down_15kV PC6 WD10_23.bmp

CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\2019 부식\19년도 부식실험 - FE-SEM 이미지\190530 - 2500H Shot-peening\190530-2\190530-2
파일명은 8-13 down_15kV PC6 WD10_16.bmp
961 993 832
이번에 띄울 이미지는  1 번째 이미지
263 264
263 265
263 266
263 268
263 269
263 270
263 271
263 273
263 275
263 277
263 279
263 281
263 283
263 285
263 289
263 291
263 293
263 295
263 298
263 301
263 302
263 304
263 306
263 308
263 310
263 312
263 314
263 316
263 318
263 320
263 321
263 324
263 325
263 328
263 332
263 333
263 335
263 338
263 340
263 342
263 344
263 347
263 349
263 351
263 352
263 355
263 356
263 358
263 359
263 360
263 361
263 363
263 363
263 364
263 365
263 365
263 366
263 367
263 367
263 368
263 369
263 370
263 370
263 371
263 371
108 557
272 273
272 274
272 274
272 275
272 275
272 278
272 279
272 281
272 281
272 282
272 285
272 285
272 286
272 289
272 290
272 293
272 294
272 297
272 298
272 301
272 302
272 305
272 306
272 309
272 310
272 311
272 312
272 313
272 314
272 316
272 316
272 3

AttributeError: 'Left' object has no attribute 'cd'

rownum 3
Data_name
8-13 down_15kV PC6 WD10_16.bmp
exshadow0 ['Data_name', '8-13 down_15kV PC6 WD10_16.bmp']
exe= 2


AttributeError: 'Left' object has no attribute 'cd'

...

CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\2019 부식\19년도 부식실험 - FE-SEM 이미지\190530 - 2500H Shot-peening\190530-2\190530-2
파일명은 8-14 up_15kV PC6 WD10_53.bmp
961 993 832
이번에 띄울 이미지는  84 번째 이미지
419 419
419 419
419 419
419 419
419 419
419 421
419 424
419 430
419 436
419 442
419 448
419 457
419 464
419 470
419 477
419 484
419 490
419 496
419 501
419 505
419 508
419 511
419 512
419 515
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 516
419 517
419 517
419 517
419 517
419 517
419 518
419 518
419 518
419 519
419 519
419 519
100 736
3.3
rownum 76
Data_name
8-13 down_15kV PC6 WD10_16.bmp
8-13 down_15kV PC6 WD10_17.bmp
8-13 down_15kV PC6 WD10_18.bmp
8-13 down_15kV PC6 WD10_19.bmp
8-13 down_15kV PC6 WD10_20.bmp
8-13 down_15kV PC6 WD10_21.bmp
8-13 down_15

413 413
413 412
413 412
413 412
413 412
413 412
413 412
413 415
413 417
413 422
413 430
413 440
413 448
413 454
413 461
413 465
413 472
413 477
413 479
413 482
413 483
413 484
413 485
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 486
413 485
413 485
413 485
413 485
413 486
413 486
413 487
413 488
413 488
413 489
413 489
413 490
413 490
413 491
413 492
413 493
413 493
413 493
413 493
413 493
413 493
413 493
413 493
413 493
413 494
413 495
413 495
82 400
5.4
rownum 77
Data_name
8-13 down_15kV PC6 WD10_16.bmp
8-13 down_15kV PC6 WD10_17.bmp
8-13 down_15kV PC6 WD10_18.bmp
8-13 down_15kV PC6 WD10_19.bmp
8-13 down_15kV PC6 WD10_20.bmp
8-13 down_15kV PC6 WD10_21.bmp
8-13 down_15kV PC6 WD10_22.bmp
8-13 down_15kV PC6 WD10_23.bmp
8-13 down_15kV PC6 WD10_24.bmp
8-13 down_15kV PC6 WD10_25.bmp
8-13 down_15kV PC6 WD10_26.bmp
8-13 down_15kV PC6 WD10_27.bmp
8-13 down_15kV PC6 WD10_28.bmp
8-13 down_15kV PC6 WD10_29.bmp
8-13 down_15kV 

8-13 down_15kV PC6 WD10_29.bmp
8-13 down_15kV PC6 WD10_30.bmp
8-13 down_15kV PC6 WD10_31.bmp
8-13 down_15kV PC6 WD10_32.bmp
8-13 down_15kV PC6 WD10_34.bmp
8-13 down_15kV PC6 WD10_35.bmp
8-13 down_15kV PC6 WD10_36.bmp
8-13 down_15kV PC6 WD10_37.bmp
8-13 down_15kV PC6 WD10_38.bmp
8-13 down_15kV PC6 WD10_39.bmp
8-13 up_15kV PC6 WD10_10.bmp
8-13 up_15kV PC6 WD10_11.bmp
8-13 up_15kV PC6 WD10_12.bmp
8-13 up_15kV PC6 WD10_13.bmp
8-13 up_15kV PC6 WD10_2.bmp
8-13 up_15kV PC6 WD10_3.bmp
8-13 up_15kV PC6 WD10_4.bmp
8-13 up_15kV PC6 WD10_5.bmp
8-13 up_15kV PC6 WD10_6.bmp
8-13 up_15kV PC6 WD10_7.bmp
8-13 up_15kV PC6 WD10_8.bmp
8-13 up_15kV PC6 WD10_9.bmp
8-14 down_15kV PC6 WD10_59.bmp
8-14 down_15kV PC6 WD10_60.bmp
8-14 down_15kV PC6 WD10_61.bmp
8-14 down_15kV PC6 WD10_62.bmp
8-14 down_15kV PC6 WD10_63.bmp
8-14 down_15kV PC6 WD10_65.bmp
8-14 down_15kV PC6 WD10_66.bmp
8-14 down_15kV PC6 WD10_67.bmp
8-14 down_15kV PC6 WD10_68.bmp
8-14 down_15kV PC6 WD10_69.bmp
8-14 down_15kV PC6 WD10_70.bmp
8-14 down

rownum 79
Data_name
8-13 down_15kV PC6 WD10_16.bmp
8-13 down_15kV PC6 WD10_17.bmp
8-13 down_15kV PC6 WD10_18.bmp
8-13 down_15kV PC6 WD10_19.bmp
8-13 down_15kV PC6 WD10_20.bmp
8-13 down_15kV PC6 WD10_21.bmp
8-13 down_15kV PC6 WD10_22.bmp
8-13 down_15kV PC6 WD10_23.bmp
8-13 down_15kV PC6 WD10_24.bmp
8-13 down_15kV PC6 WD10_25.bmp
8-13 down_15kV PC6 WD10_26.bmp
8-13 down_15kV PC6 WD10_27.bmp
8-13 down_15kV PC6 WD10_28.bmp
8-13 down_15kV PC6 WD10_29.bmp
8-13 down_15kV PC6 WD10_30.bmp
8-13 down_15kV PC6 WD10_31.bmp
8-13 down_15kV PC6 WD10_32.bmp
8-13 down_15kV PC6 WD10_34.bmp
8-13 down_15kV PC6 WD10_35.bmp
8-13 down_15kV PC6 WD10_36.bmp
8-13 down_15kV PC6 WD10_37.bmp
8-13 down_15kV PC6 WD10_38.bmp
8-13 down_15kV PC6 WD10_39.bmp
8-13 up_15kV PC6 WD10_10.bmp
8-13 up_15kV PC6 WD10_11.bmp
8-13 up_15kV PC6 WD10_12.bmp
8-13 up_15kV PC6 WD10_13.bmp
8-13 up_15kV PC6 WD10_2.bmp
8-13 up_15kV PC6 WD10_3.bmp
8-13 up_15kV PC6 WD10_4.bmp
8-13 up_15kV PC6 WD10_5.bmp
8-13 up_15kV PC6 WD10_6.bmp
8-13 up_15k

8-13 up_15kV PC6 WD10_12.bmp
8-13 up_15kV PC6 WD10_13.bmp
8-13 up_15kV PC6 WD10_2.bmp
8-13 up_15kV PC6 WD10_3.bmp
8-13 up_15kV PC6 WD10_4.bmp
8-13 up_15kV PC6 WD10_5.bmp
8-13 up_15kV PC6 WD10_6.bmp
8-13 up_15kV PC6 WD10_7.bmp
8-13 up_15kV PC6 WD10_8.bmp
8-13 up_15kV PC6 WD10_9.bmp
8-14 down_15kV PC6 WD10_59.bmp
8-14 down_15kV PC6 WD10_60.bmp
8-14 down_15kV PC6 WD10_61.bmp
8-14 down_15kV PC6 WD10_62.bmp
8-14 down_15kV PC6 WD10_63.bmp
8-14 down_15kV PC6 WD10_65.bmp
8-14 down_15kV PC6 WD10_66.bmp
8-14 down_15kV PC6 WD10_67.bmp
8-14 down_15kV PC6 WD10_68.bmp
8-14 down_15kV PC6 WD10_69.bmp
8-14 down_15kV PC6 WD10_70.bmp
8-14 down_15kV PC6 WD10_71.bmp
8-14 down_15kV PC6 WD10_72.bmp
8-14 down_15kV PC6 WD10_73.bmp
8-14 down_15kV PC6 WD10_74.bmp
8-14 down_15kV PC6 WD10_75.bmp
8-14 down_15kV PC6 WD10_76.bmp
8-14 down_15kV PC6 WD10_77.bmp
8-14 down_15kV PC6 WD10_78.bmp
8-14 down_15kV PC6 WD10_79.bmp
8-14 down_15kV PC6 WD10_80.bmp
8-14 down_15kV PC6 WD10_81.bmp
8-14 down_15kV PC6 WD10_82.bmp
8-14 